# Standard vs Structure-Constrained SINDy

Compare the performance of standard SINDy with the structure-constrained variant.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sc_sindy import (
    sindy_stls,
    sindy_structure_constrained,
    build_library_2d,
    compute_derivatives_finite_diff,
    compute_structure_metrics,
)
from sc_sindy.systems import VanDerPol

## Generate Noisy Data

In [ ]:
system = VanDerPol(mu=1.0)
t, X_clean = system.simulate([1.0, 0.0], t_span=(0, 20), dt=0.01)

# Add noise
noise_level = 0.05
X_noisy = X_clean + noise_level * np.std(X_clean) * np.random.randn(*X_clean.shape)

dt = t[1] - t[0]
X_dot = compute_derivatives_finite_diff(X_noisy, dt)

## Run Both Methods

In [ ]:
Theta, labels = build_library_2d(X_noisy)
true_xi = system.true_coefficients(labels)

# Standard SINDy
xi_standard, _ = sindy_stls(Theta, X_dot, threshold=0.1)

# Structure-Constrained SINDy
# Note: Requires trained network
try:
    xi_sc, _ = sindy_structure_constrained(
        Theta, X_dot, X_noisy,
        threshold=0.1,
        structure_threshold=0.3
    )
    has_sc = True
except Exception as e:
    print(f"SC-SINDy not available: {e}")
    has_sc = False

## Compare Results

In [ ]:
metrics_standard = compute_structure_metrics(xi_standard, true_xi)
print("Standard SINDy:")
print(f"  Precision: {metrics_standard['precision']:.3f}")
print(f"  Recall: {metrics_standard['recall']:.3f}")
print(f"  F1: {metrics_standard['f1']:.3f}")

if has_sc:
    metrics_sc = compute_structure_metrics(xi_sc, true_xi)
    print("\nStructure-Constrained SINDy:")
    print(f"  Precision: {metrics_sc['precision']:.3f}")
    print(f"  Recall: {metrics_sc['recall']:.3f}")
    print(f"  F1: {metrics_sc['f1']:.3f}")